In [ ]:
import pandas as pd
from workalendar.registry import registry
import statsmodels.formula.api as smf
import numpy as np

In [ ]:
countries=snakemake.params.countries
time_zones=pd.read_csv(snakemake.input.time_zones)

dem=pd.read_csv(snakemake.input.base_dem,index_col=0)
dem.index=pd.to_datetime(dem.index)
    
assert pd.isnull(dem).any(axis="rows").any() == False

year=2012

In [ ]:
def euro_calendars(countries):
    
    cal_replace={"AL":"RS","BA":"RS","XK":"RS","LI":"CH","MK":"GR","ME":"GR"}
    
    cals={}
    for c in countries:
        
        cal=registry.get_calendars([c])
        
        if not bool(cal):
            c2=cal_replace[c]
            cals[c]=registry.get_calendars([c2])[c2]
        else:
            cals[c]=cal[c]
            
    return cals

In [ ]:
cals=euro_calendars(countries)

In [ ]:
def dem_model(yr,cnt,d,tz,calendar):
    
    t=pd.read_csv(snakemake.input.degree_days,parse_dates=True).loc[:,["UTC",cnt,cnt+"_cdh",cnt+"_hdh"]]
    
    t["UTC"]=pd.to_datetime(t["UTC"],format="%Y-%m-%d %H:%M:%S")
    t.set_index("UTC",inplace=True)
    t.rename({cnt+"_cdh":"CDH",cnt+"_hdh":"HDH"},axis=1,inplace=True)
    
    d=d.merge(t,left_index=True,right_index=True,suffixes=("","_t2m"))

    d["month"]=d.index.to_series().dt.month
    d["dayofweek"]=d.index.to_series().dt.dayofweek
    d["hour"]=d.index.to_series().dt.hour

    d["holiday"]=0
    
    holidays = [a for (a,b) in calendar().holidays(yr)]

    d.loc[d.index.to_series().dt.date.isin(holidays),"holiday"]=1
    
    coeffs=[]
    for h in d["hour"].drop_duplicates():
        df=d.loc[d["hour"]==h,:]
        res = smf.ols(formula=cnt+' ~ HDH + CDH + C(month) + C(dayofweek) + C(holiday)', data=df).fit()
         
        coeffs.append([h,np.where(res.pvalues["HDH"]<0.01,res.params["HDH"],0), \
                           np.where(res.pvalues["CDH"]<0.01,res.params["CDH"],0),res.rsquared])

        
    coeffs=pd.DataFrame(coeffs,columns=["hour","HDH_slope","CDH_slope","rsquared"])

    #res = smf.ols(formula=cnt+' ~ HDH + CDH + C(month) + C(dayofweek) + C(hour) + C(holiday)', data=d).fit()
    #print(res.summary())

    d["heat"]=np.where(coeffs["HDH_slope"].mean() > 0.,coeffs["HDH_slope"].mean()*d["HDH"],0)
    d["cool"]=np.where(coeffs["CDH_slope"].mean() > 0.,coeffs["CDH_slope"].mean()*d["CDH"],0)

    #print(d["cool"].sum()/1E6)
    #print(d["heat"].sum()/1E6)

    d.loc[:,cnt]=d.loc[:,cnt]-d["heat"]-d["cool"]
    
    return d.loc[:,cnt].round(1),[coeffs["rsquared"].mean(), \
                         d["heat"].sum()/1E6,d["cool"].sum()/1E6]

In [ ]:
fits=[]
for cnt in countries:

    tz=time_zones.loc[time_zones["ISO2"]==cnt,"UTC"] 
    cal=cals[cnt]
        
    print(cnt)
    
    new_dem,stats=dem_model(year,cnt,dem,tz,cal)
    rsquared,tot_cool,tot_heat=stats
        
    #rsquared,phdh,pcdh=stats
        
    dem.loc[:,cnt]=new_dem
    fits.append([cnt,rsquared,tot_cool,tot_heat])
    #fits.append([cnt,rsquared,phdh,pcdh])
    
pd.DataFrame(fits,columns=["Country","rsquared","tot_heat","tot_cool"]).to_csv(snakemake.output.fit_stats,index=False)    
    
dem.to_csv(snakemake.output[0])